<h1><font color="aquamarine">K Fall Data Preprocessing</font></h1>

<h3><font color="lightblue">Importing Libraries</font></h3>

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import os


<h3><font color="lightblue">Setup Paths and Constants</font></h3>

In [3]:
sensor_train_data_path = "./datasets/sensor/sensor_data/"
sensor_train_label_path = "./datasets/sensor/label_data/"

<h3><font color="lightblue">Importing dataset</font></h3>

In [16]:
## Counting the number of samples in the entire dataset

count = 0
for folder in os.listdir(sensor_train_data_path):
    print(folder)
    folder_count = 0
    for csv_file in os.listdir(os.path.join(sensor_train_data_path,folder)):
        folder_count +=1
    print("current_total: ",count)
    print("folder_count:",folder_count)
    count = count+folder_count
print("Total: ",count)

SA06
current_total:  0
folder_count: 156
SA07
current_total:  156
folder_count: 150
SA08
current_total:  306
folder_count: 163
SA09
current_total:  469
folder_count: 153
SA10
current_total:  622
folder_count: 159
SA11
current_total:  781
folder_count: 163
SA12
current_total:  944
folder_count: 142
SA13
current_total:  1086
folder_count: 160
SA14
current_total:  1246
folder_count: 162
SA15
current_total:  1408
folder_count: 162
SA16
current_total:  1570
folder_count: 163
SA17
current_total:  1733
folder_count: 161
SA18
current_total:  1894
folder_count: 160
SA19
current_total:  2054
folder_count: 158
SA20
current_total:  2212
folder_count: 156
SA21
current_total:  2368
folder_count: 163
SA22
current_total:  2531
folder_count: 161
SA23
current_total:  2692
folder_count: 163
SA24
current_total:  2855
folder_count: 153
SA25
current_total:  3008
folder_count: 163
SA26
current_total:  3171
folder_count: 161
SA27
current_total:  3332
folder_count: 157
SA28
current_total:  3489
folder_count: 1